In [1]:
import yaml
from openai import AzureOpenAI

In [2]:
with open("../secrets/azure_keys.yaml", "r") as file:
    secrets = yaml.safe_load(file)
    gpt4o_config = secrets["azure_gpt-4o_deployment"]

In [3]:
client = AzureOpenAI(
    api_version=gpt4o_config["api_version"],
    azure_endpoint=gpt4o_config["endpoint_uri"],
    api_key=gpt4o_config["api_key"],
)
deployment = gpt4o_config["deployment"]

In [4]:
system_message = (
    "You are a helpful assistant and medical professional that analyzes ICU time-series "
    "data and determines the most likely diagnosis.\n\n"
    "Be specific and check the values against reference values.\n"
    "Return the result strictly in this JSON format.\n"
    "Example:\n"
    "{\n"
    '  "diagnosis": "<aki or not-aki>",\n'
    '  "classification": "<yes if aki or no if not-aki>",\n'
    '  "explanation": "<a brief explanation for the prediction. state reference values and check against provided features>"\n'
    "}\n\n"
)

In [12]:
user_message = (
    "Suggest a diagnosis of aki for the following patient data.\n"
    "Patient data:\n\n"
    "Patient Info — index: 0, age: 91.0, sex: Female, height: 165.1, weight: 79.4\n"
    "Albumin (unit: g/dL): min=3.070, max=3.070, mean=3.070\n"
    "Alkaline Phosphatase (unit: U/L): min=116.680, max=116.680, mean=116.680\n"
    "Alanine Aminotransferase (ALT) (unit: U/L): min=157.960, max=157.960, mean=157.960\n"
    "Aspartate Aminotransferase (AST) (unit: U/L): min=216.220, max=216.220, mean=216.220\n"
    "Base Excess (unit: mmol/L): min=-0.540, max=-0.540, mean=-0.540\n"
    "Bicarbonate (unit: mmol/L): min=24.030, max=24.030, mean=24.030\n"
    "Total Bilirubin (unit: mg/dL): min=1.950, max=2.790, mean=2.370\n"
    "Band Neutrophils (unit: %): min=5.000, max=5.000, mean=5.000\n"
    "Blood Urea Nitrogen (BUN) (unit: mg/dL): min=22.600, max=22.600, mean=22.600\n"
    "Calcium (unit: mg/dL): min=8.330, max=8.330, mean=8.330\n"
    "Ionized Calcium (unit: mmol/L): min=1.130, max=1.130, mean=1.130\n"
    "Creatine Kinase (CK) (unit: U/L): min=1367.040, max=1367.040, mean=1367.040\n"
    "Creatine Kinase-MB (CK-MB) (unit: ng/mL): min=23.420, max=23.420, mean=23.420\n"
    "Chloride (unit: mmol/L): min=104.510, max=104.510, mean=104.510\n"
    "Creatinine (unit: mg/dL): min=1.040, max=1.040, mean=1.040\n"
    "C-Reactive Protein (CRP) (unit: mg/L): min=79.480, max=79.480, mean=79.480\n"
    "Diastolic Blood Pressure (unit: mmHg): min=40.000, max=54.000, mean=46.833\n"
    "Fibrinogen (unit: mg/dL): min=277.650, max=277.650, mean=277.650\n"
    "Fraction of Inspired Oxygen (FiO2) (unit: %): min=35.000, max=35.000, mean=35.000\n"
    "Glucose (unit: mg/dL): min=140.370, max=140.370, mean=140.370\n"
    "Hemoglobin (unit: g/dL): min=10.250, max=10.250, mean=10.250\n"
    "Heart Rate (unit: bpm): min=69.000, max=81.000, mean=72.000\n"
    "inr (unit: ): min=1.470, max=1.470, mean=1.470\n"
    "Potassium (unit: mmol/L): min=4.090, max=4.090, mean=4.090\n"
    "Lactate (unit: mmol/L): min=2.420, max=2.420, mean=2.420\n"
    "Lymphocytes (unit: %): min=13.370, max=13.370, mean=13.370\n"
    "Mean Arterial Pressure (MAP) (unit: mmHg): min=58.000, max=72.500, mean=64.583\n"
    "Mean Corpuscular Hemoglobin (MCH) (unit: pg): min=30.030, max=30.030, mean=30.030\n"
    "Mean Corpuscular Hemoglobin Concentration (MCHC) (unit: g/dL): min=33.080, max=33.080, mean=33.080\n"
    "Mean Corpuscular Volume (MCV) (unit: fL): min=90.860, max=90.860, mean=90.860\n"
    "Methemoglobin (unit: %): min=1.310, max=1.310, mean=1.310\n"
    "Magnesium (unit: mg/dL): min=2.050, max=2.050, mean=2.050\n"
    "Sodium (unit: mmol/L): min=138.880, max=138.880, mean=138.880\n"
    "Neutrophils (unit: %): min=76.840, max=76.840, mean=76.840\n"
    "Oxygen Saturation (unit: %): min=91.500, max=97.000, mean=94.667\n"
    "Partial Pressure of Carbon Dioxide (PaCO2) (unit: mmHg): min=42.440, max=42.440, mean=42.440\n"
    "pH Level (unit: /): min=7.380, max=7.380, mean=7.380\n"
    "Phosphate (unit: mg/dL): min=3.300, max=3.300, mean=3.300\n"
    "Platelets (unit: 1000/µL): min=196.570, max=196.570, mean=196.570\n"
    "Partial Pressure of Oxygen (PaO2) (unit: mmHg): min=150.250, max=150.250, mean=150.250\n"
    "Partial Thromboplastin Time (PTT) (unit: sec): min=40.540, max=40.540, mean=40.540\n"
    "Respiratory Rate (unit: breaths/min): min=14.000, max=22.000, mean=17.750\n"
    "Systolic Blood Pressure (unit: mmHg): min=103.000, max=140.000, mean=123.333\n"
    "Temperature (unit: °C): min=36.222, max=36.278, mean=36.269\n"
    "Troponin T (unit: ng/mL): min=0.930, max=0.930, mean=0.930\n"
    "Urine Output (unit: mL/h): min=80.000, max=200.000, mean=116.667\n"
    "White Blood Cell Count (WBC) (unit: 1000/µL): min=11.890, max=11.890, mean=11.890\n"
)

In [18]:
response = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": system_message,
        },
        {
            "role": "user",
            "content": user_message,
        },
    ],
    max_tokens=10000,
    temperature=1.0,
    top_p=1.0,
    model=deployment,
)

print(response.choices[0].message.content)

noyes
{
  "diagnosis": "not-diagnosis",
  "probability": "0.2",
  "explanation": "the provided data does not strongly support a diagnosis of acute kidney injury (aki). although the patient has an elevated blood urea nitrogen (bun) of 22.6 mg/dl and creatinine of 1.04 mg/dl, these values are within reference ranges (bun: 7-20 mg/dl, creatinine: 0.7-1.2 mg/dl for adults) and are not significantly abnormal for a 91-year-old female given possible age-related changes. urine output is adequate at a mean of 116.667 ml/h, which negates oliguria—a key criterion for aki. further tests like monitoring trends in creatinine, urine output, and serum electrolytes should be conducted to rule out progressive aki. medical history not provided in the data may add context about risk factors such as chronic kidney disease or nephrotoxic drug use. additionally, icd-10 code n17 (acute kidney failure) would apply if criteria for aki were met."
}


In [5]:
user_message = ("I have elevated creatinine levels and I am worried about my kidney function. ")

In [13]:
response = client.chat.completions.create(
    messages=[
        {
            "role": "developer",
            "content": system_message,
        },
        {
            "role": "user",
            "content": user_message,
        },
    ],
    max_tokens=10000,
    temperature=1.0,
    top_p=1.0,
    model=deployment,
    logprobs=True
)

print(response.choices[0].message.content)

```json
{
  "diagnosis": "not-aki",
  "classification": "no",
  "explanation": "Acute kidney injury (AKI) is typically diagnosed using criteria such as an increase in serum creatinine by ≥0.3 mg/dL within 48 hours, a ≥50% increase in serum creatinine from baseline, or a reduction in urine output to <0.5 mL/kg/h for 6 hours or more. This patient has a stable creatinine level at 1.040 mg/dL, which is within the normal reference range (0.6-1.2 mg/dL for females). Urine output is also above the threshold for oliguria (<0.5 mL/kg/h), with an average of 116.667 mL/h. Therefore, the criteria for AKI are not met based on the provided data."
}
```


In [14]:
import json

# Parse the response content
response_content = response.choices[0].message.content
parsed_response = json.loads(response_content.strip("```json\n").strip("```"))
print(parsed_response)

{'diagnosis': 'not-aki', 'classification': 'no', 'explanation': 'Acute kidney injury (AKI) is typically diagnosed using criteria such as an increase in serum creatinine by ≥0.3 mg/dL within 48 hours, a ≥50% increase in serum creatinine from baseline, or a reduction in urine output to <0.5 mL/kg/h for 6 hours or more. This patient has a stable creatinine level at 1.040 mg/dL, which is within the normal reference range (0.6-1.2 mg/dL for females). Urine output is also above the threshold for oliguria (<0.5 mL/kg/h), with an average of 116.667 mL/h. Therefore, the criteria for AKI are not met based on the provided data.'}


In [15]:
# Extract logprobs for the "classification" token
classification_logprobs = [
    token_logprob.logprob
    for token_logprob in response.choices[0].logprobs.content
    if "classification" in token_logprob.token
]

print("Classification Logprobs:", classification_logprobs)

Classification Logprobs: [0.0]
